In [ ]:
import pandas as pd
from itertools import product
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import scipy.integrate as scint
import itertools
import time
from itertools import product
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm
import math
from lmfit import minimize, Parameters, Parameter, report_fit

def bee_eq(y, t, w1,w2, external_temperature, truth, peri, treat):
    hive_temperature = y
    u1 = [0]*len(hive_temperature)
    u2 = [0]*len(hive_temperature)
    for i in range(len(hive_temperature)):
        if external_temperature[i] >= 0:
            u1[i] = 1
        else:
            u2[i] = -1
    if treat == 0:
        dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature + external_temperature)
    else:
        dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature) + external_temperature + peri
    return dydt

def run_bee_eq(t, a, w1, w2, ext_temp, truth, peri, treat):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, ext_temp, truth, peri, treat), col_deriv = True, rtol = 1e-3, atol = 1e-3) # 'w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data, treat):
    #x0 = ps['a0'].value
    
    w1 = ps['w1'].value
    w2 = ps['w2'].value
    theta_ideal = ps['theta_ideal'].value
    data_date = pd.DataFrame(data).groupby(data['Date'])
    model = []
    for i,j in data_date:
        ext_temp = j[ext_data].values
        truth = j[temp_data].values
        try:
            treat = j['Treatment'].values[0]
        except:
            treat = 0
        if  treat == 0:
            peri = j[ext_data].values
        else:
            peri = j[peri_data].values
        a = [10]*len(ext_temp)
        model = np.concatenate((model,run_bee_eq(ts, a, w1, w2, ext_temp - theta_ideal,truth - theta_ideal, peri-theta_ideal, treat)))
    return (model - data[temp_data] + theta_ideal).ravel()

def main(*argv):
    filename = argv[0]
    temp_data = argv[1]
    ext_data = argv[2]
    peri_data = argv[3]
    type_ = argv[4]


    data = pd.read_csv(filename)
    params = Parameters()
    params.add('w1', value= 1.0, min=0, max=80)
    params.add('w2', value=1.0, min=0, max=80)
    params.add('theta_ideal', value = 35.0, min = 31, max = 38)
    d = pd.DataFrame(data).groupby(data['Date'])

    x_left = np.linspace(0, len(data[temp_data].values)-1, num = len(data[temp_data]))
    xx_left = data[ext_data].values
    yy_left = data[temp_data].values
    if type_ == 'Y':
        zz_left = data[peri_data].values
        treat = data['Treatment'].values[0]
    else:
        zz_left = data[temp_data].values
        treat = 0
    result = minimize(residual, params, args=(x_left, data, treat), method='leastsq')
    sig_left = yy_left + result.residual.reshape(data[temp_data].shape)
    L = np.sum(np.log(norm.pdf(data[temp_data].values.reshape(1,-1) - np.array(sig_left).reshape(1,-1))))
    pos = [0,len(d)]
    b = 1
    min_sig = [sig_left]
    all_params = [np.array([result.params['w1'].value, result.params['w2'].value, result.params['theta_ideal'].value])]
    all_positions = [pos]
    all_lks = [-2*L + 2*3]
    #print(all_lks)
    while b < len(d):
        position = []
        params_ = []
        signals = []
        likelihood = []
        for a in range(len(d)):
            if a not in pos:
                positions = np.append(pos,a)
                positions.sort()
                #print(positions)
                sig_left = []
                left_params = []
                #print(len(positions))
                #signal_left_len = 0
                for i in range(len(positions)-1):
                    params = Parameters()
                    params.add('w1', value= 1.0, min=0, max=80)
                    params.add('w2', value=1.0, min=0, max=80)
                    params.add('theta_ideal', value = 35.0, min = 31, max = 38)
                    data_position = data.groupby(data['Position'])
                    for s,t in data_position:
                        if s == positions[i]:
                            left_portion = t
                        if s > positions[i] and s < positions[i+1]:
                            left_portion = pd.concat([left_portion, t])
                    x_left = np.linspace(0, len(left_portion[temp_data].values)-1, num = len(left_portion[temp_data]))
                    xx_left =left_portion[ext_data].values
                    yy_left = left_portion[temp_data].values
                    if type_ == 'Y':
                        zz_left = left_portion[peri_data].values
                        treat = left_portion['Treatment'].values[0]
                    else:
                        zz_left = left_portion[temp_data].values
                        treat = 0
                    result = minimize(residual, params, args=(x_left, left_portion, treat), method='leastsq')
                    signal_left = yy_left + result.residual.reshape(left_portion[temp_data].shape)
                    sig_left = np.concatenate((sig_left,signal_left))
                        #print('bye')
                    left_params.append(np.array([result.params['w1'].value, result.params['w2'].value, result.params['theta_ideal'].value]))
                params_.append(left_params)
                #print(params_)
                position.append(positions)
                signals.append(sig_left)
                L = np.sum(np.log(norm.pdf(data[temp_data].values.reshape(1,-1) - np.array(sig_left).reshape(1,-1))))
                likelihood.append(-2*L + 2*((len(positions)-2)+(len(positions)-1)*3))
        predicted = signals[np.argmin(likelihood)]
        all_params.append(params_[np.argmin(likelihood)])
        #print(all_params)
        all_positions.append(position[np.argmin(likelihood)])
        all_lks.append(likelihood[np.argmin(likelihood)])
        min_sig.append(predicted)
        pos = position[np.argmin(likelihood)]
        print(pos)
        if all_lks[-1] > all_lks[-2]:
            lower = 0.1*min(all_lks)
            for m in range(len(all_lks)):
                if all_lks[m] <= min(all_lks) + lower:
                    cut_points = all_positions[m]
                    sig = min_sig[m]
                    params_ = all_params[m]
                    print(cut_points)
                    print(params_)
                    for n in sig:
                        print(n)
                    break
            break
        b = b+1
    
if __name__ == "__main__":
    hive_number = input('Enter Hive Number: ')
    hive_type = input('Is it treated? Y or N: ')
    filename = 'C_'+hive_number+'.csv'
    temp_data = 'Temp_'+ hive_number
    ext_data = 'Temp_Box'
    peri_data = 'Temp_Easy_'+ hive_number
    main(filename, temp_data, ext_data, peri_data, hive_type)